In [78]:
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize

csv_filename = 'product_details_monitor.csv'
csv_bought_together = 'items_bought_together.csv'
csv_customer_reviews = 'links_to_customer_reviews_monitor.csv'
txt_customer_reviews = 'links_to_customer_reviews_monitor.txt'

In [79]:
df = pd.read_json('monitor_product_output.jsonl')
df.head()

,name,product_summary,product_tech_spec,product_addl_info,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews
0,HP 24mh FHD Monitor - Computer Monitor with 23...,"[{'info': 'Brand', 'value': 'HP'}, {'info': 'I...","[{'info': 'Standing screen display size', 'val...","[{'info': 'Customer Reviews', 'value': '/* * F...",Moonlight LLC.,https://www.amazon.com/gp/help/seller/at-a-gla...,None,None,/HP-24mh-FHD-Monitor-Built/product-reviews/B08...
1,"Sceptre E248W-19203R 24"" Ultra Thin 75Hz 1080p...","[{'info': 'Brand', 'value': 'Sceptre'}, {'info...","[{'info': 'Standing screen display size', 'val...","[{'info': 'Customer Reviews', 'value': '/* * F...",None,None,None,None,/Sceptre-E248W-19203R-Monitor-Speakers-Metalli...
2,"Sceptre Curved 27"" 75Hz LED Monitor HDMI VGA B...","[{'info': 'Brand', 'value': 'Sceptre'}, {'info...","[{'info': 'Standing screen display size', 'val...","[{'info': 'Customer Reviews', 'value': '/* * F...",None,None,None,None,/Sceptre-Monitor-Speakers-Edge-Less-C275W-1920...
3,Sceptre IPS 27-Inch Business Computer Monitor ...,"[{'info': 'Brand', 'value': 'Sceptre'}, {'info...","[{'info': 'Standing screen display size', 'val...","[{'info': 'Customer Reviews', 'value': '/* * F...",None,None,None,None,/Sceptre-Business-Computer-Speakers-e275W-FPT/...
4,"SAMSUNG LC24F396FHNXZA 23.5"" FHD Curved LED-Li...","[{'info': 'Brand', 'value': 'SAMSUNG'}, {'info...","[{'info': 'Standing screen display size', 'val...","[{'info': 'Customer Reviews', 'value': '/* * F...",None,None,None,None,/Samsung-23-5-Inch-LED-Lit-Monitor-LC24F396FHN...


In [80]:
df = df.dropna(axis=0, how='all')

In [81]:
df.link_to_all_reviews = 'https://www.amazon.com/' + df.link_to_all_reviews

In [82]:
df.reset_index(drop=True, inplace = True)

In [83]:
# Extraction of all specifications in nested JSON-dicts to be used for building additional columns 
  
new_columns = []
for i in range(len(df)):
    if df.product_summary[i] is not None:
        product_sum = df.product_summary[i]
        x = len(product_sum)  
        for j in range(x):
            new_columns.append(product_sum[j]['info'])
    if df.product_tech_spec[i] is not None:
        product_tech = df.product_tech_spec[i]
        y = len(product_tech)  
        for j in range(y):
            new_columns.append(product_tech[j]['info'])
    if df.product_addl_info[i] is not None:
        product_add = df.product_addl_info[i]
        z = len(product_add)  
        for j in range(z):
            new_columns.append(product_add[j]['info'])

In [84]:
for i in range(len(new_columns)):
    if new_columns[i] is not None:
        new_columns[i] = new_columns[i].lower()
new_columns = set(new_columns) 

In [85]:
new_columns = list(new_columns)

In [86]:
try:
    new_columns.remove(None)
except:
    pass

In [87]:
for i in new_columns:
    df[i] = i
    df[i] = ''

In [88]:
for i in range(len(df)):
    if df.product_summary[i] is not None:
        psc = []
        product_sum = df.product_summary[i]
        product_sum = pd.DataFrame(product_sum)
        product_sum = product_sum.transpose()
        psc_len = len(product_sum.columns)  # collecting number of columns in product_sum / items in df.product_summary
        for j in range(psc_len):            # getting item / column names
            psc.append(product_sum[j][0])
        for k in range(len(psc)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if psc[k] is not None:
                psc[k] = psc[k].lower()
        product_sum.columns = product_sum.iloc[0] # Turns row 1 into column names
        product_sum = product_sum.drop('info') # Drops row 1
        product_sum.columns = psc
        for l in psc: 
            try:
                df[l][i] = product_sum[l]['value']
            except:
                continue
    if df.product_tech_spec[i] is not None:
        ptc = []
        product_tec = df.product_tech_spec[i]
        product_tec = pd.DataFrame(product_tec)
        product_tec = product_tec.transpose()
        ptc_len = len(product_tec.columns)  # collecting number of columns in product_tec / items in df.product_tech_spec
        for j in range(ptc_len):            # getting item / column names
            ptc.append(product_tec[j][0])
        for k in range(len(ptc)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if ptc[k] is not None:
                ptc[k] = ptc[k].lower()
        product_tec.columns = product_tec.iloc[0] # Turns row 1 into column names
        product_tec = product_tec.drop('info') # Drops row 1
        product_tec.columns = ptc
        for l in ptc:   ## What does this loop through??? I need the column name, but I have to stay within one row!!!
            try:
                df[l][i] = product_tec[l]['value']
            except:
                continue
    if df.product_addl_info[i] is not None:
        pac = []
        product_add = df.product_addl_info[i]
        product_add = pd.DataFrame(product_add)
        product_add = product_add.transpose()
        pac_len = len(product_add.columns)  # collecting number of columns in product_tec / items in df.product_tech_spec
        for j in range(pac_len):            # getting item / column names
            pac.append(product_add[j][0])
        for k in range(len(pac)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if pac[k] is not None:
                pac[k] = pac[k].lower()
        product_add.columns = product_add.iloc[0] # Turns row 1 into column names
        product_add = product_add.drop('info') # Drops row 1
        product_add.columns = pac
        for l in pac:   ## What does this loop through??? I need the column name, but I have to stay within one row!!!
            try:
                df[l][i] = product_add[l]['value']
            except:
                continue

In [89]:
df.rename(columns = {'asin' : 'ASIN'}, inplace = True)

In [90]:
for i in range(len(df)):
    if df['customer reviews'][i] is not None:
        try:
            res = [re.findall(r'(\d+).(\d+) out of 5 stars', df['customer reviews'][i])[0] ]
            a = res[0][0]
            b = res[0][1]
            c = float(a+'.'+b)
            df['customer reviews'][i] = c
        except:
            pass
df.rename(columns = {'customer reviews' : 'rating'}, inplace = True)

In [91]:
df = df.drop(['product_summary', 'product_tech_spec', 'product_addl_info'], axis = 1)

In [92]:
for i in range(len(df)):
    if df.name[i] is None:
        df = df.drop([i], axis = 0)

In [93]:
# Creating dataframe with links to all customer reviews for saving in additional CSV-file

df_monitor_customer_reviews_link = df.link_to_all_reviews
df_monitor_customer_reviews_link = df_monitor_customer_reviews_link.dropna()
df_monitor_customer_reviews_link.reset_index(drop=True, inplace = True)

In [94]:
df_monitor_bought_together = df[['ASIN', 'name', 'freq_bought', 'freq_bought_link']]

In [95]:
df.to_csv(csv_filename)
df_monitor_customer_reviews_link.to_csv(csv_customer_reviews) # actually not necessary anymore since there's being a txt-file created
df_monitor_bought_together.to_csv(csv_bought_together)

In [96]:
f = open(txt_customer_reviews, 'w+')

for i in range(len(df_monitor_customer_reviews_link)):
    f.write(df_monitor_customer_reviews_link[i])
    f.write('\n')

f.close()

In [97]:
df2 = pd.read_csv(csv_filename)
df2

,Unnamed: 0,name,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews,assembled width,average life,hard drive interface,...,color name,part number,shade color,certification,light direction,rating,shade material,os,product dimensions,item weight
0,0,HP 24mh FHD Monitor - Computer Monitor with 23...,Moonlight LLC.,https://www.amazon.com/gp/help/seller/at-a-gla...,NaN,NaN,https://www.amazon.com//HP-24mh-FHD-Monitor-Bu...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.7,NaN,NaN,23.54 x 8.33 x 18.28 inches,9.94 pounds
1,1,"Sceptre E248W-19203R 24"" Ultra Thin 75Hz 1080p...",NaN,NaN,NaN,NaN,https://www.amazon.com//Sceptre-E248W-19203R-M...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.6,NaN,NaN,21.59 x 7.02 x 15.83 inches,5.77 pounds
2,2,"Sceptre Curved 27"" 75Hz LED Monitor HDMI VGA B...",NaN,NaN,NaN,NaN,https://www.amazon.com//Sceptre-Monitor-Speake...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.7,NaN,NaN,24.27 x 9.01 x 17.69 inches,8.6 pounds
3,3,Sceptre IPS 27-Inch Business Computer Monitor ...,NaN,NaN,NaN,NaN,https://www.amazon.com//Sceptre-Business-Compu...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.7,NaN,NaN,24.12 x 7.03 x 17.12 inches,7.12 pounds
4,4,"SAMSUNG LC24F396FHNXZA 23.5"" FHD Curved LED-Li...",NaN,NaN,NaN,NaN,https://www.amazon.com//Samsung-23-5-Inch-LED-...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.7,NaN,NaN,8.61 x 21.56 x 16.68 inches,6.8 pounds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4980,4986,KabelDirekt – 6ft – 4K HDMI cable (4K@120Hz & ...,KabelDirekt Products,https://www.amazon.com/gp/help/seller/at-a-gla...,HDMI Switch 4K HDMI Splitter-Techole Aluminum ...,https://www.amazon.com/HDMI-Switch-4K-Splitter...,https://www.amazon.com//Cable-supports-FullHD-...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4981,4987,OSD Audio TSM-T9 Home and Office Dual Arm Desk...,NaN,NaN,NaN,NaN,https://www.amazon.com//OSD-Audio-13-inch-23-i...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4982,4988,MSI Gaming GeForce GT 710 2GB GDRR3 64-bit HDC...,NaN,NaN,NaN,NaN,https://www.amazon.com//MSI-GT-710-2GD3-LP/pro...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.4,NaN,NaN,5.75 x 0.75 x 2.72 inches,5.1 ounces
4983,4989,MMOBIEL Heavy Duty Suction Cups 2 Pcs Compatib...,Menko Teleco,https://www.amazon.com/gp/help/seller/at-a-gla...,LeFix Replacement LCD Panel Adhesive Tape Stri...,https://www.amazon.com/Replacement-Panel-Adhes...,https://www.amazon.com//MMOBIEL-Suction-Compat...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
